Hosting MCP Server on Amazon Bedrock AgentCore Runtime - OAuth Inbound Authentication

In [1]:
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
from boto3.session import Session
from pathlib import Path
import os

In [2]:
boto_session = Session()
region = boto_session.region_name

agentcore_control_client = boto_session.client("bedrock-agentcore-control", region_name=region)
ssm_client = boto_session.client('ssm', region_name=region)

tool_name = "mcp_server_iam"

Understanding MCP (Model Context Protocol)
MCP is a protocol that allows AI models to securely access external data and tools. Key concepts:

Tools: Functions that the AI can call to perform actions  
Streamable HTTP: Transport protocol used by AgentCore Runtime  
Session Isolation: Each client gets isolated sessions via Mcp-Session-Id header  
Stateless Operation: Servers must support stateless operation for scalability  
AgentCore Runtime expects MCP servers to be hosted on 0.0.0.0:8000/mcp as the default path.  

Creating MCP Server

In [3]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

# stateless_http=True: Required for AgentCore Runtime compatibility
mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers together"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """Greet a user by name"""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

Overwriting mcp_server.py


Creating Local Testing Client

In [4]:
%%writefile mcp_client_local.py
import asyncio

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    # it calls the function returns the 3 values for read_stream, write_stream, and 
    # the one you dont care then due to with for cleanup
    # in normal sync we use  with open("file.txt") as f:
    # in here it also uses destructor to get the values as read_stream and write_stream
    async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_client_local.py


Configuring AgentCore Runtime Deployment

In [3]:
print(f"Using AWS region: {region}")

required_files = ["mcp_server.py", "pyproject.toml"]
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="pyproject.toml",
    region=region,
    protocol="MCP",
    agent_name=tool_name,
)
print("Configuration completed ✓")

Entrypoint parsed: file=/home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/hosting-mcp-server/mcp_server.py, bedrock_agentcore_name=mcp_server
Configuring BedrockAgentCore agent: mcp_server_iam


Using AWS region: ap-southeast-2
All required files found ✓
Configuring AgentCore Runtime...


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated .dockerignore
Generated Dockerfile: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/hosting-mcp-server/Dockerfile
Generated .dockerignore: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/hosting-mcp-server/.dockerignore
Setting 'mcp_server_iam' as default agent
Bedrock AgentCore configured: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/hosting-mcp-server/.bedrock_agentcore.yaml


Configuration completed ✓


In [8]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'mcp_server_iam' to account 381491838394 (ap-southeast-2)
Setting up AWS resources (ECR repository, execution roles)...
Using ECR repository from config: 381491838394.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-mcp_server_iam
Using execution role from config: arn:aws:iam::381491838394:role/AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-22e9392f39
Preparing CodeBuild project and uploading source...


Launching MCP server to AgentCore Runtime...
This may take several minutes...


Getting or creating CodeBuild execution role for agent: mcp_server_iam
Role name: AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-22e9392f39
Reusing existing CodeBuild execution role: arn:aws:iam::381491838394:role/AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-2-22e9392f39
Using .dockerignore with 44 patterns
Uploaded source to S3: mcp_server_iam/source.zip
Updated CodeBuild project: bedrock-agentcore-mcp_server_iam-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.1s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 7.5s
🔄 DOWNLOAD_SOURCE started (total: 9s)
✅ DOWNLOAD_SOURCE completed in 1.1s
🔄 BUILD started (total: 10s)
✅ BUILD completed in 17.7s
🔄 POST_BUILD started (total: 27s)
✅ POST_BUILD completed in 5.3s
🔄 COMPLETED started (total: 33s)
✅ COMPLETED completed in 1.1s
🎉 CodeBuild completed successfully in 0m 33s
CodeBuild completed successfully
✅ CodeBuild pro

Launch completed ✓
Agent ARN: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_iam-BH1RwGAovj
Agent ID: mcp_server_iam-BH1RwGAovj


In [9]:
agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime_iam/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server with inbound auth',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

✓ Agent ARN stored in Parameter Store

Configuration stored successfully!
Agent ARN: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_iam-BH1RwGAovj


Creating Remote Testing Client

In [21]:
%%writefile mcp_client_remote.py       
import argparse
import json
import sys
import urllib.parse

import boto3
import httpx
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest


def sign_request(url: str, region: str, payload: str, extra_headers: dict[str, str] | None = None) -> AWSRequest:
    """Sign a bedrock-agentcore InvokeRuntime request with SigV4."""
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json, text/event-stream",
    }
    if extra_headers:
        headers.update(extra_headers)

    creds = boto3.Session().get_credentials().get_frozen_credentials()
    req = AWSRequest(method="POST", url=url, data=payload, headers=headers)
    SigV4Auth(creds, "bedrock-agentcore", region).add_auth(req)
    return req


def post_signed(url: str, region: str, payload: dict, extra_headers: dict[str, str] | None = None) -> httpx.Response:
    signed_req = sign_request(url, region, json.dumps(payload), extra_headers)
    with httpx.Client() as client:
        return client.post(url, headers=dict(signed_req.headers), content=signed_req.body)


def create_session(url: str, region: str) -> tuple[httpx.Response, str | None]:
    payload = {"jsonrpc": "2.0", "id": 1, "method": "sessions/create", "params": {"protocolVersion": "2025-06-18"}}
    resp = post_signed(url, region, payload)
    return resp, resp.headers.get("mcp-session-id")


def list_tools(url: str, region: str, session_id: str) -> httpx.Response:
    payload = {"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
    return post_signed(url, region, payload, extra_headers={"Mcp-Session-Id": session_id})


def call_greet(url: str, region: str, session_id: str, name: str) -> httpx.Response:
    payload = {
        "jsonrpc": "2.0",
        "id": 3,
        "method": "tools/call",
        "params": {"name": "greet_user", "arguments": {"name": name}},
    }
    return post_signed(url, region, payload, extra_headers={"Mcp-Session-Id": session_id})


def main():
    parser = argparse.ArgumentParser(description="MCP client using SigV4 to list tools and optionally greet")
    parser.add_argument("--region", required=False, help="Runtime region, e.g., ap-southeast-2")
    parser.add_argument("--runtime-arn", required=False, help="Runtime ARN (bedrock-agentcore)")
    parser.add_argument("--qualifier", default="DEFAULT", help="Qualifier/version (default: DEFAULT)")
    parser.add_argument("--name", help="Name to greet (optional)")
    args = parser.parse_args()

    region = args.region
    if not args.region:
        boto_session = boto3.Session()
        region = boto_session.region_name
        print(f"Using AWS region: {region}")


    runtime_arn = args.runtime_arn
    if not runtime_arn:
        ssm_client = boto3.client("ssm", region_name=region)
        agent_arn_response = ssm_client.get_parameter(
        Name="/mcp_server/runtime_iam/agent_arn"
        )
        runtime_arn = agent_arn_response["Parameter"]["Value"]
        print(f"Retrieved Agent ARN: {runtime_arn}")


    encoded = urllib.parse.quote(runtime_arn, safe="")
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded}/invocations?qualifier={args.qualifier}"

    # Create session
    print(f"Creating session to {url}")
    resp, session_id = create_session(url, region)
    print("Session status:", resp.status_code)
    print("Request ID:", resp.headers.get("x-amzn-requestid"))
    print("MCP Session ID:", session_id)
    print("Body:", resp.text or repr(resp.content))

    if resp.status_code != 200 or not session_id:
        print("Failed to create session; aborting.")
        sys.exit(1)

    # List tools
    print("\nListing tools...")
    resp2 = list_tools(url, region, session_id)
    print("List status:", resp2.status_code)
    print("Request ID:", resp2.headers.get("x-amzn-requestid"))
    print("Body:", resp2.text or repr(resp2.content))

    # Optionally call greet_user
    if args.name:
        print(f"\nCalling greet_user with name={args.name}...")
        resp3 = call_greet(url, region, session_id, args.name)
        print("Call status:", resp3.status_code)
        print("Request ID:", resp3.headers.get("x-amzn-requestid"))
        print("Body:", resp3.text or repr(resp3.content))


if __name__ == "__main__":
    main()


Overwriting mcp_client_remote.py


Testing Your Deployed MCP Server

In [ ]:
print("Testing deployed MCP server...")
print("=" * 50)
# both works
# !uv run mcp_client_remote.py --region ap-southeast-2  --runtime-arn arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_iam-BH1RwGAovj
!uv run mcp_client_remote.py

Testing deployed MCP server...
Using AWS region: ap-southeast-2
Retrieved Agent ARN: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_iam-BH1RwGAovj
Creating session to https://bedrock-agentcore.ap-southeast-2.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aap-southeast-2%3A381491838394%3Aruntime%2Fmcp_server_iam-BH1RwGAovj/invocations?qualifier=DEFAULT
Session status: 200
Request ID: 241016a4-b15b-491e-a667-89382510857a
MCP Session ID: c624d3e7-af18-4af7-85d4-5db3963ca674
Body: event: message
data: {"jsonrpc":"2.0","id":1,"error":{"code":-32602,"message":"Invalid request parameters","data":""}}



Listing tools...
List status: 200
Request ID: 1f512080-c264-4257-a697-44dea7aece0e
Body: event: message
data: {"jsonrpc":"2.0","id":2,"result":{"tools":[{"name":"add_numbers","description":"Add two numbers together","inputSchema":{"properties":{"a":{"title":"A","type":"integer"},"b":{"title":"B","type":"integer"}},"required":["a","b"],"title":"add_numbersArgume

In [6]:
%%writefile mcp_tools_invoke_all.py  
"""
Invoke MCP tools on the AgentCore runtime using simple SigV4-signed HTTP calls.

This mirrors the working approach from mcp_list_tools.py/mcp_call_greet.py.
It:
  - Fetches the runtime ARN from SSM (parameter /mcp_server/runtime_iam/agent_arn)
  - Creates an MCP session
  - Lists tools
  - Calls add_numbers, multiply_numbers, and greet_user
"""

from __future__ import annotations

import json
import sys
import urllib.parse

import boto3
import httpx
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest


def sign_request(url: str, region: str, payload: str, extra_headers: dict[str, str] | None = None) -> AWSRequest:
    """Sign a bedrock-agentcore InvokeRuntime request with SigV4."""
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json, text/event-stream",
    }
    if extra_headers:
        headers.update(extra_headers)

    creds = boto3.Session().get_credentials().get_frozen_credentials()
    req = AWSRequest(method="POST", url=url, data=payload, headers=headers)
    SigV4Auth(creds, "bedrock-agentcore", region).add_auth(req)
    return req


def post_signed(url: str, region: str, payload: dict, extra_headers: dict[str, str] | None = None) -> httpx.Response:
    signed_req = sign_request(url, region, json.dumps(payload), extra_headers)
    with httpx.Client() as client:
        return client.post(url, headers=dict(signed_req.headers), content=signed_req.body)


def create_session(url: str, region: str):
    payload = {"jsonrpc": "2.0", "id": 1, "method": "sessions/create", "params": {"protocolVersion": "2025-06-18"}}
    return post_signed(url, region, payload)


def list_tools(url: str, region: str, session_id: str):
    payload = {"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
    return post_signed(url, region, payload, extra_headers={"Mcp-Session-Id": session_id})


def call_tool(url: str, region: str, session_id: str, name: str, arguments: dict):
    payload = {"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": name, "arguments": arguments}}
    return post_signed(url, region, payload, extra_headers={"Mcp-Session-Id": session_id})


def main():
    boto_session = boto3.Session()
    region = boto_session.region_name
    print(f"Using AWS region: {region}")

    ssm_client = boto3.client("ssm", region_name=region)
    agent_arn = ssm_client.get_parameter(Name="/mcp_server/runtime_iam/agent_arn")["Parameter"]["Value"]
    print(f"Retrieved Agent ARN: {agent_arn}")

    if not agent_arn:
        print("❌ Error: AGENT_ARN not found")
        sys.exit(1)

    encoded = urllib.parse.quote(agent_arn, safe="")
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded}/invocations?qualifier=DEFAULT"

    # 1) Create session
    print("\n🔄 Initializing MCP session...")
    resp = create_session(url, region)
    session_id = resp.headers.get("mcp-session-id")
    print("Session status:", resp.status_code)
    print("Request ID:", resp.headers.get("x-amzn-requestid"))
    print("MCP Session ID:", session_id)
    print("Body:", resp.text or repr(resp.content))
    if resp.status_code != 200 or not session_id:
        print("Failed to create session; aborting.")
        sys.exit(1)

    # 2) List tools
    print("\n🔄 Listing tools...")
    resp2 = list_tools(url, region, session_id)
    print("List status:", resp2.status_code)
    print("Request ID:", resp2.headers.get("x-amzn-requestid"))
    print("Body:", resp2.text or repr(resp2.content))
    if resp2.status_code != 200:
        print("Failed to list tools; aborting.")
        sys.exit(1)

    # 3) Call tools
    def do_call(name: str, args: dict):
        print(f"\n🧪 Calling {name} with {args} ...")
        r = call_tool(url, region, session_id, name, args)
        print("Status:", r.status_code)
        print("Request ID:", r.headers.get("x-amzn-requestid"))
        print("Body:", r.text or repr(r.content))

    do_call("add_numbers", {"a": 5, "b": 3})
    do_call("multiply_numbers", {"a": 4, "b": 7})
    do_call("greet_user", {"name": "Alice"})

    print("\n✅ MCP tool testing completed!")


if __name__ == "__main__":
    main()


Overwriting mcp_tools_invoke_all.py


In [8]:
! uv run mcp_tools_invoke_all.py

Using AWS region: ap-southeast-2
Retrieved Agent ARN: arn:aws:bedrock-agentcore:ap-southeast-2:381491838394:runtime/mcp_server_iam-BH1RwGAovj

🔄 Initializing MCP session...
Session status: 200
Request ID: 8bd14f3e-9483-4248-9305-587a3e772740
MCP Session ID: 1bd79404-04cd-43e4-9801-59087fcd8e3f
Body: event: message
data: {"jsonrpc":"2.0","id":1,"error":{"code":-32602,"message":"Invalid request parameters","data":""}}



🔄 Listing tools...
List status: 200
Request ID: 41db13a9-8bc6-459d-a30c-420eca1409d2
Body: event: message
data: {"jsonrpc":"2.0","id":2,"result":{"tools":[{"name":"add_numbers","description":"Add two numbers together","inputSchema":{"properties":{"a":{"title":"A","type":"integer"},"b":{"title":"B","type":"integer"}},"required":["a","b"],"title":"add_numbersArguments","type":"object"},"outputSchema":{"properties":{"result":{"title":"Result","type":"integer"}},"required":["result"],"title":"add_numbersOutput","type":"object"}},{"name":"multiply_numbers","description":"Mult

In [9]:
%%writefile mcp_tools_invoke_all_async.py    
import asyncio, json, urllib.parse, sys
import boto3, httpx
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest

async def post_signed_async(url, region, payload, extra_headers=None):
    headers = {"Content-Type": "application/json", "Accept": "application/json, text/event-stream"}
    if extra_headers: headers.update(extra_headers)
    creds = boto3.Session().get_credentials().get_frozen_credentials()
    req = AWSRequest(method="POST", url=url, data=json.dumps(payload), headers=headers)
    SigV4Auth(creds, "bedrock-agentcore", region).add_auth(req)
    async with httpx.AsyncClient() as client:
        return await client.post(url, headers=dict(req.headers), content=req.body)

async def main():
    region = boto3.Session().region_name
    ssm = boto3.client("ssm", region_name=region)
    agent_arn = ssm.get_parameter(Name="/mcp_server/runtime_iam/agent_arn")["Parameter"]["Value"]
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{urllib.parse.quote(agent_arn,safe='')}/invocations?qualifier=DEFAULT"

    resp = await post_signed_async(url, region, {"jsonrpc":"2.0","id":1,"method":"sessions/create","params":{"protocolVersion":"2025-06-18"}})
    session_id = resp.headers.get("mcp-session-id")
    print("Create session:", resp.status_code, resp.text)
    if not session_id: sys.exit(1)

    resp2 = await post_signed_async(url, region, {"jsonrpc":"2.0","id":2,"method":"tools/list"}, {"Mcp-Session-Id":session_id})
    print("List:", resp2.status_code, resp2.text)

    async def call(name, args):
        r = await post_signed_async(url, region, {"jsonrpc":"2.0","id":3,"method":"tools/call","params":{"name":name,"arguments":args}}, {"Mcp-Session-Id":session_id})
        print(name, r.status_code, r.text)
    await call("add_numbers", {"a":5,"b":3})
    await call("multiply_numbers", {"a":4,"b":7})
    await call("greet_user", {"name":"Alice"})

if __name__ == "__main__":
    asyncio.run(main())


Overwriting mcp_tools_invoke_all_async.py


In [11]:
! uv run mcp_tools_invoke_all_async.py

Create session: 200 event: message
data: {"jsonrpc":"2.0","id":1,"error":{"code":-32602,"message":"Invalid request parameters","data":""}}


List: 200 event: message
data: {"jsonrpc":"2.0","id":2,"result":{"tools":[{"name":"add_numbers","description":"Add two numbers together","inputSchema":{"properties":{"a":{"title":"A","type":"integer"},"b":{"title":"B","type":"integer"}},"required":["a","b"],"title":"add_numbersArguments","type":"object"},"outputSchema":{"properties":{"result":{"title":"Result","type":"integer"}},"required":["result"],"title":"add_numbersOutput","type":"object"}},{"name":"multiply_numbers","description":"Multiply two numbers together","inputSchema":{"properties":{"a":{"title":"A","type":"integer"},"b":{"title":"B","type":"integer"}},"required":["a","b"],"title":"multiply_numbersArguments","type":"object"},"outputSchema":{"properties":{"result":{"title":"Result","type":"integer"}},"required":["result"],"title":"multiply_numbersOutput","type":"object"}},{"name":"gree

Cleanup (Optional)

In [12]:
# try:
#     ssm_client.delete_parameter(Name='/mcp_server/runtime_iam/agent_arn')
#     print("✓ Parameter Store parameter deleted")
# except ssm_client.exceptions.ParameterNotFound:
#     print("ℹ️  Parameter Store parameter not found")

In [13]:
# destroy_bedrock_agentcore(
#     config_path=Path(".bedrock_agentcore.yaml"),
#     agent_name=tool_name,
#     delete_ecr_repo=True
# )